## Setup

In [1]:
%run utils.ipynb

import time
import pickle

import gpflow
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

from ordinal_likelihood import Ordinal

W0818 18:45:05.554438 4611364288 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/session_manager.py:31: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0818 18:45:05.559500 4611364288 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/misc.py:27: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0818 18:45:05.779114 4611364288 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/training/tensorflow_optimizer.py:169: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0818 18:45:05.780936 4611364288 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/training/tensorflow_optimizer.py:156: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1

In [2]:
# Constants
datasets = {'agg': {'x': {'cont':         x_cont_agg, 
                          'disc':         x_disc_agg}, 
                    'y': {# 'abs':          y_abs_agg['y'],
                          # 'abs_binned':   y_abs_agg['binned_y'],
                          'share':        y_share_agg['y'],
                          'share_binned': y_share_agg['binned_y']}
                   },
            'ind': {'x': {'cont':         x_cont_ind,
                          'disc':         x_disc_ind},
                    'y': {# 'abs':          y_abs_ind,
                          # 'abs_binned':   y_abs_bin_ind,
                          'share':        y_share_ind,
                          'share_binned': y_share_bin_ind}
                   }
           }

kernels = ['Matern12', 'Matern32', 'Matern52', 'RBF']

agg_level = 'both'

## Functions

In [3]:
def rec_dd():
        return defaultdict(rec_dd)

In [4]:
def get_kern(kern):
    if kern == 'Matern12': return gpflow.kernels.Matern12(1)
    if kern == 'Matern32': return gpflow.kernels.Matern32(1) 
    if kern == 'Matern52': return gpflow.kernels.Matern52(1)
    if kern == 'RBF':      return gpflow.kernels.RBF(1)

In [5]:
# create x and y arrays
def x_and_y(x, y):
    change = {'decrease':     2,
              'constant':     1, 
              'increase':     0,
              'fewer':        2,
              'same':         1,
              'more':         0,
              'not_increase': 1}
    x = np.array(x)
    y = np.array(pd.Series(y).replace(change).values.astype('int64')).reshape(y.shape[0], 1)
    return x, y

In [6]:
# get bin edges and likelihood
def create_likelihood(y):
    bin_edges = np.array(np.arange(np.unique(y).size + 1), dtype=float)
    # Need to check in on this, tutorial does the below which ends up with negative bins
    # bin_edges = bin_edges - bin_edges.mean()
    bin_edges = bin_edges - .5
    return Ordinal(bin_edges)

In [15]:
# build a model with this likelihood
def build_model(x_train, y_train, likelihood, kernel):
    gaussian_model = gpflow.models.VGP(tf.cast(x_train, tf.float64),
                                       y_train, 
                                       kern=kernel,
                                       likelihood=likelihood)
    # fit the model
    gpflow.train.ScipyOptimizer().minimize(gaussian_model)
    return gaussian_model

In [8]:
# get predictive densities
def test_model(model, x_test):
    densities = []
    # Predictive density for a single input x
    for x in x_test:
        ys = np.arange(np.max(model.Y.value+1)).reshape([-1, 1])
        x_new_vec = x*np.ones_like(ys)
        # for predict_density x and y need to have the same number of rows
        dens = np.exp(model.predict_density(x_new_vec, ys))
        densities.append(dens)
    return densities

In [9]:
# get accuracy
def accuracy(y_test, densities):
    score = 0
    for index, y in enumerate(y_test):
        if y == np.argmax(densities[index]): score += 1
    return score/len(y_test)

In [ ]:
# multiclass ROC
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [10]:
# figure out the mse
def scale_pred(pred):
    factor = 1/sum(pred)
    scaled = []
    for p in pred:
        scaled.append(p * factor)
    return tuple(scaled)

def calc_errors(x_test_all, densities, agg_level, dist):
    mapping = {0: 'increase',
               1: 'constant/not increase',
               2: 'decrease'}
    y_true = []
    y_pred = []
    true_class = []
    pred_class = []
    # true then pred
    confusion = defaultdict(lambda: defaultdict(int))
    for index, x in enumerate(x_test_all.iterrows()):
        x = tuple(x[1].values)
        pred = tuple([p for [p] in densities[index]])
        pred = scale_pred(pred)
        if agg_level == 'ind': x = tuple(list(x[1:]) + [x[0]])
        true_info = noc_dict[dist][x]
        noc = true_info['noc']
        if len(pred) == 3: true_dist = true_info['share']
        else:              true_dist = true_info['share_binned']
        y_true.append(true_dist)
        y_pred.append(pred)
        y_t = np.argmax(true_dist)
        y_p = np.argmax(pred)
        true_class.append(y_t)
        pred_class.append(y_p)
        confusion[mapping[y_t]][mapping[y_p]] += 1
        
    mse = mean_squared_error(y_true, y_pred)
    roc = multiclass_roc_auc_score(true_class, pred_class)
    return mse, roc, confusion

In [11]:
# do cross validation
def cv(x, y, x_all, y_all, kern, agg_level, dist):
    if agg_level == 'ind': x_all = x_all.drop(['noc', 'participant.id'], axis=1)
    kf = KFold(n_splits=5)
    scores = []
    results = []
    mse = []
    roc = []
    total_confusion = defaultdict(lambda: defaultdict(int))

    for train_index, test_index in kf.split(x):
        start = time.time()
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_test_all = x_all.iloc[test_index]
        kernel = get_kern(kern)
        likelihood = create_likelihood(y_all)
        model = build_model(x_train, y_train, likelihood, kernel)
        densities = test_model(model, x_test)
        scores.append(accuracy(y_test, densities))
        mse_error, roc_error, confusion = calc_errors(x_test_all, densities, agg_level, dist)
        mse.append(mse_error)
        roc.append(roc_error)
        for y_t in confusion:
            for y_p in confusion[y_t]:
                total_confusion[y_t][y_p] += confusion[y_t][y_p]
        results.append(create_results(x_test, densities))
        end = time.time()
        print('Fold tested in (sec):', end - start)
        del kernel, likelihood, model

    return {'Avg. score':  sum(scores)/len(scores),
            'Avg. mse':    sum(mse)/len(mse),
            'Avg. roc':    sum(roc)/len(roc),
            'Confusion':   dict(total_confusion),
            'All scores':  scores,
            'All mse':     mse,
            'All roc':     roc,
            'All results': results}

In [12]:
def pickle_results(results, params):
    with open('gp_results/{}.pkl'.format(params), 'wb') as f:
        pickle.dump(results, f)

In [13]:
def organize_results(results):
    best_result = {'Matern12': ['', 0],
                   'Matern32': ['', 0],
                   'Matern52': ['', 0],
                   'RBF':      ['', 0]}

    for kern in results:
        for agg_name in results[kern]:
            for var_type in results[kern][agg_name]:
                for result_dist in results[kern][agg_name][var_type]:
                    params = ' '.join([kern, agg_name, var_type, result_dist])
                    score = results[kern][agg_name][var_type][result_dist]['Avg. score']
                    if score == best_result[kern][1]: 
                        best_result[kern][0] += ', ' + params
                    if score > best_result[kern][1]: 
                        best_result[kern] = [params, score]
        
    return best_result

In [14]:
def create_results(x_test, densities):
    results = {}
    for index, x in enumerate(x_test):
        results[(tuple(x))] = densities[index]
    return results